In [17]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [18]:
header_info = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Whale/3.24.223.18 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,ko;q=0.8,ko-KR;q=0.7"
    # "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,ko-KR;q=0.7"
}
url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"

response = requests.get(url, headers=header_info)

In [19]:
html = response.text.replace("\r", "").replace("\n", "").replace("\t", "")
soup = BeautifulSoup(html, "html.parser")

In [20]:
li_list = soup.select('li.ipc-metadata-list-summary-item.cli-parent')
len(li_list)

250

In [21]:
top_250_movies = []
for li in li_list:
    title = li.select_one("h3").text.strip()
    dot_idx = title.find(".") # "1. 쇼생크 탈출"에서 .의 위치 찾기
    rank, title = title[:dot_idx], title[dot_idx + 1:].strip() # 순위와 제목으로 분할해서 저장

    spans = li.select('span.cli-title-metadata-item')
    metadata_info = [span.text.strip() for span in spans]
    # metadata_info = metadata_info if len(metadata_info) == 3 else metadata_info + ["-"]
    if len(metadata_info) < 3:
        metadata_info += ["-"]*(3-len(metadata_info))

    # rating, vote_count = li.select_one('.cli-ratings-container span:first-child').text.strip().split("\xa0")
    rating = li.select_one('.cli-ratings-container span:first-child').text.strip().split("\xa0")[0] # "\xa0" : &nbsp;
    top_250_movies.append((rank, title, *metadata_info, rating))



In [ ]:
# [ "1", "title", *["1972", "2h 22m", "15"], "9.3" ] --> [ "1", "title", "1972", "2h 22m", "15", "9.3" ]

In [22]:
top_250_movies

[('1', '쇼생크 탈출', '1994', '2h 22m', '15', '9.3'),
 ('2', 'Daeboo', '1972', '2h 55m', '18', '9.2'),
 ('3', '다크 나이트', '2008', '2h 32m', '15', '9.0'),
 ('4', 'The Godfather Part II', '1974', '3h 22m', '18', '9.0'),
 ('5', '12명의 성난 사람들', '1957', '1h 36m', '15', '9.0'),
 ('6', '쉰들러 리스트', '1993', '3h 15m', '15', '9.0'),
 ('7', '반지의 제왕: 왕의 귀환', '2003', '3h 21m', '12', '9.0'),
 ('8', '펄프 픽션', '1994', '2h 34m', '18', '8.9'),
 ('9', '반지의 제왕: 반지 원정대', '2001', '2h 58m', '12', '8.9'),
 ('10', '석양의 무법자', '1966', '2h 41m', '12', '8.8'),
 ('11', '포레스트 검프', '1994', '2h 22m', '12', '8.8'),
 ('12', '파이트 클럽', '1999', '2h 19m', '18', '8.8'),
 ('13',
  'The Lord of the Rings: The Two Towers',
  '2002',
  '2h 59m',
  '12',
  '8.8'),
 ('14', '인셉션', '2010', '2h 28m', '12', '8.8'),
 ('15', '스타워즈 에피소드 5 - 제국의 역습', '1980', '2h 4m', 'All', '8.7'),
 ('16', '매트릭스', '1999', '2h 16m', '12', '8.7'),
 ('17', '좋은 친구들', '1990', '2h 25m', '18', '8.7'),
 ('18', '뻐꾸기 둥지 위로 날아간 새', '1975', '2h 13m', '18', '8.7'),
 ('19', '세븐',

In [23]:
import oracledb

In [24]:
oracledb.init_oracle_client()

In [14]:
# 데이터베이스 연결
conn = oracledb.connect(user="kdtda2", password="kdtda2", dsn="127.0.0.1:1521/xe") # user-id, password, dbms-url

# 명령(SQL) 실행
cursor = conn.cursor()

cursor.execute( """create table top250movies
                   ( ranking number(3,0) primary key, 
                     title varchar2(100 char), 
                     year_of_release number(4,0), 
                     running_time varchar2(10 byte),
                     age_restricted varchar2(20 byte),
                     rating number(2, 1) ) """)


# 연결 종료
conn.close()

In [15]:
# 데이터베이스 연결
conn = oracledb.connect(user="kdtda2", password="kdtda2", dsn="127.0.0.1:1521/xe") # user-id, password, dbms-url

# 명령(SQL) 실행
cursor = conn.cursor()

cursor.executemany( """insert into top250movies values(:1, :2, :3, :4, :5, :6)""", top_250_movies)
cursor.execute("commit")

# 연결 종료
conn.close()